In [4]:
!pip3 install textblob

     |████████████████████████████████| 645kB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 10.4MB/s eta 0:00:01
  Running setup.py install for nltk ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


# Generating Author CSV

In [2]:
import re
import time
import codecs
import urllib.request
import pandas as pd
import numpy as np
from PIL import Image
import bs4 as bs
import html
import gensim
import json
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn as sns
from IPython.display import display, HTML
from gensim.utils import simple_preprocess
from textblob import TextBlob
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
def read_custom_csv(path,col_sep,row_sep):
    with open(path,'r') as custom_csv:
        lines = re.split(row_sep, custom_csv.read())
    h_text = lines[0].split()[0]
    header_list = h_text.split(col_sep)
    if row_sep == r"\|\|\|":
        lines[0] = lines[0][len(h_text):]
    else:
        lines[0] = lines[0][len(h_text) + len(row_sep) - 1:]
    headers={h.strip() : [] for h in header_list}
    for line in lines:
        cols = line[1:].split(col_sep)
        if len(cols) == len(header_list):
            for i in range(len(header_list)):
                headers[header_list[i]].append(cols[i])
        elif len(cols) == 1:
            continue
        else:
            print('invalid csv')
            break
    return pd.DataFrame(headers)

In [3]:
authors_1 = pd.read_csv('./output/Iteration_1/authors.csv')
authors_2 = pd.read_csv('./output/Iteration_2/authors.csv')
authors_text1 = read_custom_csv('./output/Iteration_2/authors_text.txt',":::",r"\n(?=.\d{5,})")
authors_text2 = read_custom_csv('./output/Iteration_3/authors_text.txt',":::",r"\n(?=.\d{5,})")
authors_text3 = read_custom_csv('./output/Iteration_4/authors_text.txt',":::",r"\n(?=.\d{5,})")

In [4]:
authors_text1['author_id'] = authors_text1['author_id'].apply(lambda x: int(x))
authors_text1['project_id'] = authors_text1['project_id'].apply(lambda x: int(x))
authors_text_base = authors_text1.copy().drop(['author_id'],axis=1)
print(authors_text_base.dtypes)
authors_text_base

project_id     int64
author_bio    object
dtype: object


,project_id,author_bio
0,682031137,Everette Hartsoe started his career in 1991 at...
1,24098319,Blood Luxury are a metal band from Tasmania.
2,980834458,"I'm an avid reader, writer, and have worked as..."
3,908929425,A professor of mathematics at SUNY College at ...
4,1211032216,Aaron Myers-Brooks is a guitarist/composer/edu...
...,...,...
2666,2095196620,"Hi there, my name is Shea and I'm the creator ..."
2667,1002385458,Cornell '19 Computer Science major with a pass...
2668,331021887,"Hello, I'm a pin designer and new pin creator...."
2669,886800902,TJ has a degree in Finance and Economics as we...


In [10]:
authors_meta1 = pd.concat([authors_1, authors_2])
print(authors_meta1.dtypes)
authors_meta1

author_id             int64
project_id            int64
author_name          object
bio_length            int64
bio_words             int64
bio_polarity        float64
bio_subjectivity    float64
author_created        int64
author_backed         int64
author_join_year      int64
dtype: object


,author_id,project_id,author_name,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year
0,1823553610,682031137,Everette Hartsoe,2534,419,0.131699,0.423469,26,47,2010
1,852506449,24098319,Blood Luxury,44,8,0.000000,0.000000,1,0,2019
2,563681582,980834458,Scott Taylor,935,161,0.233333,0.533333,55,77,2011
3,543968884,908929425,Joshua Palmatier,826,138,0.100000,0.331250,9,58,2013
4,1067103572,1211032216,Aaron Myers-Brooks,312,41,-0.016667,0.075000,1,2,2013
...,...,...,...,...,...,...,...,...,...,...
2199,273851621,2095196620,Shea Knight,166,31,0.237500,0.283333,7,59,2018
2200,1822157202,1002385458,Cindy Wang,61,10,0.062500,0.500000,1,0,2019
2201,227325954,331021887,LittleLion,165,32,0.193182,0.383523,1,2,2019
2202,2059751528,886800902,TJ Smith,544,86,0.076420,0.406818,1,2,2018


In [11]:
author_info1 = pd.merge(authors_meta1, authors_text_base, on="project_id")
author_info1

,author_id,project_id,author_name,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year,author_bio
0,1823553610,682031137,Everette Hartsoe,2534,419,0.131699,0.423469,26,47,2010,Everette Hartsoe started his career in 1991 at...
1,852506449,24098319,Blood Luxury,44,8,0.000000,0.000000,1,0,2019,Blood Luxury are a metal band from Tasmania.
2,563681582,980834458,Scott Taylor,935,161,0.233333,0.533333,55,77,2011,"I'm an avid reader, writer, and have worked as..."
3,543968884,908929425,Joshua Palmatier,826,138,0.100000,0.331250,9,58,2013,A professor of mathematics at SUNY College at ...
4,1067103572,1211032216,Aaron Myers-Brooks,312,41,-0.016667,0.075000,1,2,2013,Aaron Myers-Brooks is a guitarist/composer/edu...
...,...,...,...,...,...,...,...,...,...,...,...
2645,273851621,2095196620,Shea Knight,166,31,0.237500,0.283333,7,59,2018,"Hi there, my name is Shea and I'm the creator ..."
2646,1822157202,1002385458,Cindy Wang,61,10,0.062500,0.500000,1,0,2019,Cornell '19 Computer Science major with a pass...
2647,227325954,331021887,LittleLion,165,32,0.193182,0.383523,1,2,2019,"Hello, I'm a pin designer and new pin creator...."
2648,2059751528,886800902,TJ Smith,544,86,0.076420,0.406818,1,2,2018,TJ has a degree in Finance and Economics as we...


In [12]:
authors_text2['author_id'] = authors_text2['author_id'].apply(lambda x: int(x))
authors_text2['project_id'] = authors_text2['project_id'].apply(lambda x: int(x))
authors_text3['author_id'] = authors_text3['author_id'].apply(lambda x: int(x))
authors_text3['project_id'] = authors_text3['project_id'].apply(lambda x: int(x))
print(authors_text2.dtypes.all() == authors_text3.dtypes.all())
author_info2 = pd.concat([authors_text2, authors_text3]).reset_index(drop=True)
author_info2

True


,author_id,project_id,author_name,author_bio,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year
0,51764633,1701293492,Terry Robinson,My name is Terry and I am a college student ma...,194,37,-0.09375,0.25,1,0,2019|||
1,38757115,316354263,Kevin Stamps,Dedicated father of two loving daughters. An i...,91,17,0.3,0.475,1,0,2019|||
2,467328161,1386713854,Darren Koziol,Darren Koziol is the creator of DARK OZ comics...,994,171,0.11546401515151515,0.4009185606060607,6,21,"2016|||"""
3,1468710437,1383984340,Wildebeest,Wildebeest is a small studio led by founder an...,317,55,0.0703125,0.3953125,1,0,"2017|||"""
4,5505576,868578023,Danny Machado,http://www.imdb.com/name/nm6771868/,35,2,0.0,0.0,3,7,2012|||
...,...,...,...,...,...,...,...,...,...,...,...
12742,655445174,2126959419,Tim Hippmann,Media Artist & Designer. Living and working wi...,270,39,0.13142857142857142,0.2542857142857143,14,96,"2016|||"""
12743,586384454,337599179,Cassandra Michael,"I have a 9 year track-record in marketing, sal...",894,149,0.11542884199134198,0.3070887445887446,1,0,"2018|||"""
12744,2106362391,239406594,"Louis, Marc & Burak","Hi guys, we are Louis, Marc & Burak. We are lo...",371,62,0.24999999999999997,0.3708333333333334,3,10,"2016|||"""
12745,1548073300,1918600366,Ally,"We decided to invent a new generation of toys,...",139,26,0.2102272727272727,0.3939393939393939,2,7,"2018|||"""


In [13]:
author_info2.columns

Index(['author_id', 'project_id', 'author_name', 'author_bio', 'bio_length',
       'bio_words', 'bio_polarity', 'bio_subjectivity', 'author_created',
       'author_backed', 'author_join_year'],
      dtype='object')

In [14]:
all_author_info = pd.concat([author_info1, author_info2]).reset_index(drop=True)[author_info2.columns]
all_author_info['author_bio']=all_author_info['author_bio'].apply(lambda x: ' '.join(simple_preprocess(x)))
all_author_info['bio_length']=all_author_info['bio_length'].apply(lambda x: int(re.sub('\D', '', str(x))) if x else np.nan)
all_author_info['bio_words']=all_author_info['bio_words'].apply(lambda x: int(x))
all_author_info['bio_polarity']=all_author_info['author_bio'].apply(lambda x: TextBlob(x).sentences[0].sentiment.polarity if x else np.nan)
all_author_info['bio_subjectivity']=all_author_info['author_bio'].apply(lambda x: TextBlob(x).sentences[0].sentiment.subjectivity if x else np.nan)
all_author_info['author_join_year'] = all_author_info['author_join_year'].apply(lambda x: int(re.sub('\D', '', str(x))))
all_author_info

,author_id,project_id,author_name,author_bio,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year
0,1823553610,682031137,Everette Hartsoe,everette hartsoe started his career in at the ...,2534.0,419,0.245395,0.501475,26,47,2010
1,852506449,24098319,Blood Luxury,blood luxury are metal band from tasmania,44.0,8,0.000000,0.000000,1,0,2019
2,563681582,980834458,Scott Taylor,an avid reader writer and have worked as senio...,935.0,161,0.316667,0.670455,55,77,2011
3,543968884,908929425,Joshua Palmatier,professor of mathematics at suny college at on...,826.0,138,0.141667,0.433333,9,58,2013
4,1067103572,1211032216,Aaron Myers-Brooks,aaron myers brooks is guitarist composer educa...,312.0,41,-0.066667,0.300000,1,2,2013
...,...,...,...,...,...,...,...,...,...,...,...
15392,655445174,2126959419,Tim Hippmann,media artist designer living and working with ...,270.0,39,0.228571,0.485714,14,96,2016
15393,586384454,337599179,Cassandra Michael,have year track record in marketing sales comm...,894.0,149,0.129972,0.365057,1,0,2018
15394,2106362391,239406594,"Louis, Marc & Burak",hi guys we are louis marc burak we are longtim...,371.0,62,0.395455,0.615909,3,10,2016
15395,1548073300,1918600366,Ally,we decided to invent new generation of toys ma...,139.0,26,0.253788,0.429293,2,7,2018


In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
new_author_info = all_author_info.copy()
new_author_info.drop(['author_name','author_bio'],axis=1,inplace=True)
new_author_info[['bio_length','bio_words','bio_polarity','bio_subjectivity','author_created','author_backed']] = scaler.fit_transform(new_author_info[['bio_length','bio_words','bio_polarity','bio_subjectivity','author_created','author_backed']])
new_author_info = pd.get_dummies(new_author_info, columns=['author_join_year'])
new_author_info

,author_id,project_id,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year_2009,author_join_year_2010,author_join_year_2011,author_join_year_2012,author_join_year_2013,author_join_year_2014,author_join_year_2015,author_join_year_2016,author_join_year_2017,author_join_year_2018,author_join_year_2019
0,1823553610,682031137,0.274510,0.280643,0.622698,0.501475,0.287356,0.034815,0,1,0,0,0,0,0,0,0,0,0
1,852506449,24098319,0.004767,0.005358,0.500000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,1
2,563681582,980834458,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,1,0,0,0,0,0,0,0,0
3,543968884,908929425,0.089481,0.092431,0.570833,0.433333,0.091954,0.042963,0,0,0,0,1,0,0,0,0,0,0
4,1067103572,1211032216,0.033799,0.027461,0.466667,0.300000,0.000000,0.001481,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15392,655445174,2126959419,0.029249,0.026122,0.614286,0.485714,0.149425,0.071111,0,0,0,0,0,0,0,1,0,0,0
15393,586384454,337599179,0.096848,0.099799,0.564986,0.365057,0.000000,0.000000,0,0,0,0,0,0,0,0,0,1,0
15394,2106362391,239406594,0.040191,0.041527,0.697727,0.615909,0.022989,0.007407,0,0,0,0,0,0,0,1,0,0,0
15395,1548073300,1918600366,0.015058,0.017415,0.626894,0.429293,0.011494,0.005185,0,0,0,0,0,0,0,0,0,1,0


In [16]:
new_author_info.to_csv('all_project_author_info_no_extra.csv',index=False)

In [9]:
author_list = all_author_info.copy()[['author_id','author_created']]
author_list['author_id'] = author_list['author_id'].apply(lambda x: 'https://www.kickstarter.com/profile/'+str(x)+'/created')
author_list

,author_id,author_created
0,https://www.kickstarter.com/profile/1823553610...,26
1,https://www.kickstarter.com/profile/852506449/...,1
2,https://www.kickstarter.com/profile/563681582/...,55
3,https://www.kickstarter.com/profile/543968884/...,9
4,https://www.kickstarter.com/profile/1067103572...,1
...,...,...
15392,https://www.kickstarter.com/profile/655445174/...,14
15393,https://www.kickstarter.com/profile/586384454/...,1
15394,https://www.kickstarter.com/profile/2106362391...,3
15395,https://www.kickstarter.com/profile/1548073300...,2


In [10]:
author_list.to_csv('authors_to_scrape.csv')

In [11]:
output = pd.read_csv('./authors_to_scrape.csv').drop(['Unnamed: 0'],axis=1

,author_id,author_created
0,https://www.kickstarter.com/profile/1823553610...,26
1,https://www.kickstarter.com/profile/852506449/...,1
2,https://www.kickstarter.com/profile/563681582/...,55
3,https://www.kickstarter.com/profile/543968884/...,9
4,https://www.kickstarter.com/profile/1067103572...,1
...,...,...
15392,https://www.kickstarter.com/profile/655445174/...,14
15393,https://www.kickstarter.com/profile/586384454/...,1
15394,https://www.kickstarter.com/profile/2106362391...,3
15395,https://www.kickstarter.com/profile/1548073300...,2


In [ ]:
import json
def pretty_print(jsondata):
    print(json.dumps(jsondata, indent=4)) 

### Scraping Extra Author Specific Information

In [ ]:
import math
import random
import logging
from num2words import num2words
res = {}
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename="./author_data_scraper.log", 
                    level=logging.DEBUG,
                    format='%(levelname)s: %(asctime)s %(message)s', 
                    filemode='a+') 
logger = logging.getLogger() 
while True:
    print("Attempting...")
    for idx in range(2150,len(output)):
        try:
            project_list=[]
            max_page = math.ceil(int(output.loc[idx]['author_created']) / 32)
            for i in range(1,max_page+1):
                time.sleep(random.randint(1,3))
                author_url = output.loc[idx]['author_id']
                html_text = urllib.request.urlopen(author_url + '?page=' + str(i)).read()   #url).read()
                soup = bs.BeautifulSoup(html_text,'lxml')
                projects = soup.find("div", {"id":"react-profile-created"})['data-projects']
                projects_object = codecs.decode(html.unescape(projects), 'unicode_escape')
                ids = re.findall('\"id\":(\d+),\"photo\"',projects_object)
                funded = re.findall('\"percent_funded\":(.*?),',projects_object)
                state = [g[1].strip('"') for g in re.findall('\"goal\":(.*?),.*?\"state\":(.*?),',projects_object)]
                category = re.findall('\"category\".*?\"slug\":\"(.*?)",',projects_object)
                for j in range(len(ids)):
                    proj_dict={}
                    proj_dict["id"]=int(ids[j])
                    proj_dict["state"]=state[j]
                    proj_dict["percent_funded"]=float(funded[j])
                    proj_dict["category"]=category[j]
                    project_list.append(proj_dict)
            scripts = soup.find_all('script')
            graph_data = scripts[10]
            result = re.sub(r"(.*)circle_data = (.*);(.*)", "\\2", str(graph_data), 0, re.MULTILINE | re.DOTALL)
            result_obj = json.loads(result) 
            creator_id = re.findall('profile/(.*?)/',author_url)[0]
            catback_dict = {}
            for j in result_obj:
                catback_dict[j['label']]=j['projects_backed']
            res[creator_id] = {
                    "created_projects" : project_list,
                    "backed_category" : catback_dict
                }
            to_save = json.dumps(res)
            f = open("extra_author_data.json","w")
            f.write(to_save)
            soup.decompose()
            f.close()
            printer_text = 'Done extracting extra data for '+num2words(idx+1 ,to='ordinal_num')+" url with author id: "+creator_id
            logger.info(printer_text)
            time.sleep(random.randint(1,5))
        except Exception as e: 
            logger.error("An Error Occured while parsing "+ num2words(idx+1 ,to='ordinal_num') + ' url: ' + str(e) + '.')
            continue
        finally:
            time.sleep(2)

In [20]:
author_info_read = pd.read_csv('./all_project_author_info_no_extra.csv')
author_info_read

,author_id,project_id,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year_2009,author_join_year_2010,author_join_year_2011,author_join_year_2012,author_join_year_2013,author_join_year_2014,author_join_year_2015,author_join_year_2016,author_join_year_2017,author_join_year_2018,author_join_year_2019
0,1823553610,682031137,0.274510,0.280643,0.622698,0.501475,0.287356,0.034815,0,1,0,0,0,0,0,0,0,0,0
1,852506449,24098319,0.004767,0.005358,0.500000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,1
2,563681582,980834458,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,1,0,0,0,0,0,0,0,0
3,543968884,908929425,0.089481,0.092431,0.570833,0.433333,0.091954,0.042963,0,0,0,0,1,0,0,0,0,0,0
4,1067103572,1211032216,0.033799,0.027461,0.466667,0.300000,0.000000,0.001481,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15392,655445174,2126959419,0.029249,0.026122,0.614286,0.485714,0.149425,0.071111,0,0,0,0,0,0,0,1,0,0,0
15393,586384454,337599179,0.096848,0.099799,0.564986,0.365057,0.000000,0.000000,0,0,0,0,0,0,0,0,0,1,0
15394,2106362391,239406594,0.040191,0.041527,0.697727,0.615909,0.022989,0.007407,0,0,0,0,0,0,0,1,0,0,0
15395,1548073300,1918600366,0.015058,0.017415,0.626894,0.429293,0.011494,0.005185,0,0,0,0,0,0,0,0,0,1,0


In [21]:
author_info_read.columns
author_info_read = author_info_read[['project_id', 'author_id', 'bio_length', 'bio_words', 'bio_polarity',
       'bio_subjectivity', 'author_created', 'author_backed',
       'author_join_year_2009', 'author_join_year_2010',
       'author_join_year_2011', 'author_join_year_2012',
       'author_join_year_2013', 'author_join_year_2014',
       'author_join_year_2015', 'author_join_year_2016',
       'author_join_year_2017', 'author_join_year_2018',
       'author_join_year_2019']]
author_info_read

,project_id,author_id,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year_2009,author_join_year_2010,author_join_year_2011,author_join_year_2012,author_join_year_2013,author_join_year_2014,author_join_year_2015,author_join_year_2016,author_join_year_2017,author_join_year_2018,author_join_year_2019
0,682031137,1823553610,0.274510,0.280643,0.622698,0.501475,0.287356,0.034815,0,1,0,0,0,0,0,0,0,0,0
1,24098319,852506449,0.004767,0.005358,0.500000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,1
2,980834458,563681582,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,1,0,0,0,0,0,0,0,0
3,908929425,543968884,0.089481,0.092431,0.570833,0.433333,0.091954,0.042963,0,0,0,0,1,0,0,0,0,0,0
4,1211032216,1067103572,0.033799,0.027461,0.466667,0.300000,0.000000,0.001481,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15392,2126959419,655445174,0.029249,0.026122,0.614286,0.485714,0.149425,0.071111,0,0,0,0,0,0,0,1,0,0,0
15393,337599179,586384454,0.096848,0.099799,0.564986,0.365057,0.000000,0.000000,0,0,0,0,0,0,0,0,0,1,0
15394,239406594,2106362391,0.040191,0.041527,0.697727,0.615909,0.022989,0.007407,0,0,0,0,0,0,0,1,0,0,0
15395,1918600366,1548073300,0.015058,0.017415,0.626894,0.429293,0.011494,0.005185,0,0,0,0,0,0,0,0,0,1,0


In [87]:
all_project_info = pd.read_csv('./all_project_info.csv')
all_project_info

,project_id,main_category_comics,main_category_crafts,main_category_dance,main_category_design,main_category_fashion,main_category_film & video,main_category_food,main_category_games,main_category_journalism,...,end_month,end_quarter_Q2,end_quarter_Q3,end_quarter_Q4,end_quarter_Q5,spotlight,staff_pick,goal,usd_pledged,state
0,682031137,1,0,0,0,0,0,0,0,0,...,0.545455,0,1,0,0,1,0,0.000006,2.009670e-04,1
1,24098319,0,0,0,0,0,0,0,0,0,...,0.545455,0,1,0,0,0,0,0.000006,1.889263e-09,0
2,980834458,0,0,0,0,0,0,0,1,0,...,0.545455,0,1,0,0,1,0,0.000017,3.256876e-04,1
3,908929425,0,0,0,0,0,0,0,0,0,...,0.545455,0,1,0,0,0,0,0.000056,2.370657e-04,0
4,1211032216,0,0,0,0,0,0,0,0,0,...,0.545455,0,1,0,0,1,0,0.000002,1.766284e-05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15210,2126959419,0,0,0,0,0,0,0,0,0,...,0.909091,0,0,1,0,1,1,0.000028,7.543276e-04,1
15211,337599179,0,0,0,0,1,0,0,0,0,...,1.000000,0,0,0,1,0,0,0.000111,5.516931e-06,0
15212,239406594,0,0,0,1,0,0,0,0,0,...,1.000000,0,0,0,1,1,0,0.000278,2.621161e-03,1
15213,1918600366,0,0,0,1,0,0,0,0,0,...,0.909091,0,0,1,0,1,0,0.000167,1.479880e-03,1


In [89]:
all_project_author_basic_info = pd.merge(author_info_read,all_project_info,on="project_id")
all_project_author_basic_info

,project_id,author_id,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year_2009,author_join_year_2010,...,end_month,end_quarter_Q2,end_quarter_Q3,end_quarter_Q4,end_quarter_Q5,spotlight,staff_pick,goal,usd_pledged,state
0,682031137,1823553610,0.274510,0.280643,0.622698,0.501475,0.287356,0.034815,0,1,...,0.545455,0,1,0,0,1,0,0.000006,2.009670e-04,1
1,24098319,852506449,0.004767,0.005358,0.500000,0.000000,0.000000,0.000000,0,0,...,0.545455,0,1,0,0,0,0,0.000006,1.889263e-09,0
2,980834458,563681582,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,...,0.545455,0,1,0,0,1,0,0.000017,3.256876e-04,1
3,908929425,543968884,0.089481,0.092431,0.570833,0.433333,0.091954,0.042963,0,0,...,0.545455,0,1,0,0,0,0,0.000056,2.370657e-04,0
4,1211032216,1067103572,0.033799,0.027461,0.466667,0.300000,0.000000,0.001481,0,0,...,0.545455,0,1,0,0,1,0,0.000002,1.766284e-05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15210,2126959419,655445174,0.029249,0.026122,0.614286,0.485714,0.149425,0.071111,0,0,...,0.909091,0,0,1,0,1,1,0.000028,7.543276e-04,1
15211,337599179,586384454,0.096848,0.099799,0.564986,0.365057,0.000000,0.000000,0,0,...,1.000000,0,0,0,1,0,0,0.000111,5.516931e-06,0
15212,239406594,2106362391,0.040191,0.041527,0.697727,0.615909,0.022989,0.007407,0,0,...,1.000000,0,0,0,1,1,0,0.000278,2.621161e-03,1
15213,1918600366,1548073300,0.015058,0.017415,0.626894,0.429293,0.011494,0.005185,0,0,...,0.909091,0,0,1,0,1,0,0.000167,1.479880e-03,1


In [132]:
all_project_author_basic_info.columns

Index(['project_id', 'author_id', 'bio_length', 'bio_words', 'bio_polarity',
       'bio_subjectivity', 'author_created', 'author_backed',
       'author_join_year_2009', 'author_join_year_2010',
       ...
       'launched_quarter_Q5', 'end_month', 'end_quarter_Q2', 'end_quarter_Q3',
       'end_quarter_Q4', 'end_quarter_Q5', 'spotlight', 'staff_pick', 'goal',
       'usd_pledged'],
      dtype='object', length=328)

In [92]:
all_project_info_rewards = pd.read_csv('./all_proj_info_rewards.csv')
all_project_info_rewards = all_project_info_rewards[['project_id','dominant_topic_Hard Intangibles','dominant_topic_Hard Tangibles','dominant_topic_Soft Intangibles','dominant_topic_Soft Tangibles','state']]
all_project_info_rewards

,project_id,dominant_topic_Hard Intangibles,dominant_topic_Hard Tangibles,dominant_topic_Soft Intangibles,dominant_topic_Soft Tangibles,state
0,682031137,0.00,0.136364,0.000000,0.000000,1
1,24098319,0.00,0.030303,0.059406,0.000000,0
2,980834458,0.05,0.030303,0.000000,0.000000,1
3,908929425,0.00,0.181818,0.000000,0.000000,0
4,1211032216,0.00,0.000000,0.000000,0.035088,1
...,...,...,...,...,...,...
15210,2126959419,0.00,0.136364,0.000000,0.000000,1
15211,337599179,0.00,0.000000,0.009901,0.105263,0
15212,239406594,0.00,0.000000,0.089109,0.000000,1
15213,1918600366,0.02,0.000000,0.000000,0.122807,1


In [93]:
all_project_author_basic_info.drop(['state'],axis=1,inplace=True)
all_project_author_reward_info = pd.merge(all_project_author_basic_info,all_project_info_rewards,on="project_id")
all_project_author_reward_info

,project_id,author_id,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year_2009,author_join_year_2010,...,end_quarter_Q5,spotlight,staff_pick,goal,usd_pledged,dominant_topic_Hard Intangibles,dominant_topic_Hard Tangibles,dominant_topic_Soft Intangibles,dominant_topic_Soft Tangibles,state
0,682031137,1823553610,0.274510,0.280643,0.622698,0.501475,0.287356,0.034815,0,1,...,0,1,0,0.000006,2.009670e-04,0.00,0.136364,0.000000,0.000000,1
1,24098319,852506449,0.004767,0.005358,0.500000,0.000000,0.000000,0.000000,0,0,...,0,0,0,0.000006,1.889263e-09,0.00,0.030303,0.059406,0.000000,0
2,980834458,563681582,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,...,0,1,0,0.000017,3.256876e-04,0.05,0.030303,0.000000,0.000000,1
3,908929425,543968884,0.089481,0.092431,0.570833,0.433333,0.091954,0.042963,0,0,...,0,0,0,0.000056,2.370657e-04,0.00,0.181818,0.000000,0.000000,0
4,1211032216,1067103572,0.033799,0.027461,0.466667,0.300000,0.000000,0.001481,0,0,...,0,1,0,0.000002,1.766284e-05,0.00,0.000000,0.000000,0.035088,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15210,2126959419,655445174,0.029249,0.026122,0.614286,0.485714,0.149425,0.071111,0,0,...,0,1,1,0.000028,7.543276e-04,0.00,0.136364,0.000000,0.000000,1
15211,337599179,586384454,0.096848,0.099799,0.564986,0.365057,0.000000,0.000000,0,0,...,1,0,0,0.000111,5.516931e-06,0.00,0.000000,0.009901,0.105263,0
15212,239406594,2106362391,0.040191,0.041527,0.697727,0.615909,0.022989,0.007407,0,0,...,1,1,0,0.000278,2.621161e-03,0.00,0.000000,0.089109,0.000000,1
15213,1918600366,1548073300,0.015058,0.017415,0.626894,0.429293,0.011494,0.005185,0,0,...,0,1,0,0.000167,1.479880e-03,0.02,0.000000,0.000000,0.122807,1


In [102]:
extra_author_info = pd.read_csv('./all_extra_570_author_info.csv')
extra_author_info.rename(columns={'state':'projects_successful'},inplace=True)
extra_author_info

,author_id,projects_successful,created_category_art,created_category_comics,created_category_crafts,created_category_dance,created_category_design,created_category_fashion,created_category_film & video,created_category_food,...,backed_Fashion,backed_Film & Video,backed_Food,backed_Games,backed_Journalism,backed_Music,backed_Photography,backed_Publishing,backed_Technology,backed_Theater
0,482895457,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1500790819,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1651606516,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,106225196,2,0,0,0,0,0,0,0,0,...,0,0,0,19,0,0,0,1,0,0
4,1883048737,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,173453060,6,4,0,1,0,0,1,0,0,...,19,0,0,1,0,0,0,0,0,0
566,545039739,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
567,1708185841,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
568,2631428,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
extra_author_info[extra_author_info.columns[1:]] = scaler.fit_transform(extra_author_info[extra_author_info.columns[1:]])
extra_author_info

,author_id,projects_successful,created_category_art,created_category_comics,created_category_crafts,created_category_dance,created_category_design,created_category_fashion,created_category_film & video,created_category_food,...,backed_Fashion,backed_Film & Video,backed_Food,backed_Games,backed_Journalism,backed_Music,backed_Photography,backed_Publishing,backed_Technology,backed_Theater
0,482895457,0.018519,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
1,1500790819,0.018519,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.5,...,0.000000,0.0,0.125,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
2,1651606516,0.000000,0.000000,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,...,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
3,106225196,0.037037,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000,0.032423,0.0,0.000000,0.0,0.037037,0.0,0.0
4,1883048737,0.037037,0.029412,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,173453060,0.111111,0.117647,0.0,0.2,0.0,0.0,0.090909,0.0,0.0,...,0.271429,0.0,0.000,0.001706,0.0,0.000000,0.0,0.000000,0.0,0.0
566,545039739,0.037037,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000,0.001706,0.0,0.000000,0.0,0.000000,0.0,0.0
567,1708185841,0.018519,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000,0.000000,0.0,0.105263,0.0,0.000000,0.0,0.0
568,2631428,0.000000,0.000000,0.0,0.0,0.0,0.0,0.181818,0.0,0.0,...,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0


In [128]:
extra_author_info.query('author_id == 1823553610')

,author_id,projects_successful,created_category_art,created_category_comics,created_category_crafts,created_category_dance,created_category_design,created_category_fashion,created_category_film & video,created_category_food,...,backed_Fashion,backed_Film & Video,backed_Food,backed_Games,backed_Journalism,backed_Music,backed_Photography,backed_Publishing,backed_Technology,backed_Theater
1,1500790819,0.018519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
len(all_project_author_reward_info['author_id']

15215

In [106]:
all_project_author_reward_info['author_id'].nunique()

14014

In [113]:
all_project_extra_author_reward_info = pd.merge(all_project_author_reward_info, extra_author_info, on="author_id")
all_project_extra_author_reward_info

,project_id,author_id,bio_length,bio_words,bio_polarity,bio_subjectivity,author_created,author_backed,author_join_year_2009,author_join_year_2010,...,backed_Fashion,backed_Film & Video,backed_Food,backed_Games,backed_Journalism,backed_Music,backed_Photography,backed_Publishing,backed_Technology,backed_Theater
0,980834458,563681582,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,...,0.014286,0.307692,0.0,0.076792,0.0,0.000000,0.0,0.296296,0.035714,0.0
1,335574025,563681582,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,...,0.014286,0.307692,0.0,0.076792,0.0,0.000000,0.0,0.296296,0.035714,0.0
2,1320460803,563681582,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,...,0.014286,0.307692,0.0,0.076792,0.0,0.000000,0.0,0.296296,0.035714,0.0
3,273320183,563681582,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,...,0.014286,0.307692,0.0,0.076792,0.0,0.000000,0.0,0.296296,0.035714,0.0
4,1887800704,563681582,0.101289,0.107837,0.658333,0.670455,0.620690,0.057037,0,0,...,0.014286,0.307692,0.0,0.076792,0.0,0.000000,0.0,0.296296,0.035714,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,230463014,173453060,0.036616,0.042197,0.616667,0.466667,0.057471,0.038519,0,0,...,0.271429,0.000000,0.0,0.001706,0.0,0.000000,0.0,0.000000,0.000000,0.0
677,1557406864,545039739,0.032608,0.032820,0.537879,0.155303,0.011494,0.000741,0,0,...,0.000000,0.000000,0.0,0.001706,0.0,0.000000,0.0,0.000000,0.000000,0.0
678,2105195374,1708185841,0.032608,0.027461,0.900000,0.950000,0.000000,0.001481,0,0,...,0.000000,0.000000,0.0,0.000000,0.0,0.105263,0.0,0.000000,0.000000,0.0
679,1318382546,2631428,0.036074,0.041527,0.472222,0.411111,0.011494,0.000000,0,0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0


<br>
<br><br><br><br><br><br><br><br><br><br><br><br><br>

In [28]:
input_size = 103
output_size = 1

In [29]:
def possible_mlp_configurations(input_size, output_size, step=10,layers=3, truncate=0, shuffle=False):
    import random
    import itertools
    minim = step
    maxim = 2*(input_size - output_size)
    neurons = tuple(range(minim,int(round(maxim,-1)),step))
    configs = [x for x in itertools.product(neurons,repeat=layers) if x[0] != x[1] and x[0] > x[1]]
    if shuffle:
        random.shuffle(configs)
    if truncate > 0:
        configs = configs[:truncate]
    return (configs)
possible_mlp_configurations(input_size, output_size, step=30, layers=2, truncate=10, shuffle=True)

[(60, 30),
 (180, 150),
 (120, 30),
 (90, 30),
 (150, 90),
 (150, 30),
 (120, 90),
 (180, 120),
 (150, 120),
 (180, 90)]